In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/rand_iu_1000_3/"

"../../graphs/rand_iu_1000_3/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [9]:
tree = []
for i in 1:5
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [10]:
x = readFromFile(graphp * "x.vec");

In [11]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [12]:
numIts = 200;

for i in 4:5
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.402588 seconds (275.40 k allocations: 11.697 MB, 24.29% gc time)
  0.178703 seconds (123.30 k allocations: 5.059 MB)
  0.861043 seconds (3.17 M allocations: 95.499 MB, 3.08% gc time)
Working on iteration 10 with a-norm error 6.645573288441024
Working on iteration 20 with a-norm error 2.5816766375651037
Working on iteration 30 with a-norm error 0.8082675130038449
Working on iteration 40 with a-norm error 0.3076513001636457
Working on iteration 50 with a-norm error 0.1313171162885011
Working on iteration 60 with a-norm error 0.04472276891904837
Working on iteration 70 with a-norm error 0.016717118214829883
Working on iteration 80 with a-norm error 0.005723144104164551
Working on iteration 90 with a-norm error 0.0018508699180618017
Working on iteration 100 with a-norm error 0.0008068750489977344
Working on iteration 110 with a-norm error 0.00031286520819973247
Working on iteration 120 with a-norm error 0.00016348824852520494
Working on iteration 130 with a-norm error 8.720245081447285